In [59]:
import pandas as pd
import numpy as np
import helper_function as hf
# %load helper_function.py
%run helper_function.py 

pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.width', 10000)

In [60]:
final_df, noc = prepare_olympic_dataset("athlete_events.csv", "noc_regions.csv")

In [40]:
sport_dict = {'Rugby': True
       'Alpinism':False, 'Speed Skating':False, 'Ice Hockey':True, 'Nordic Combined':False,
       'Rhythmic Gymnastics':False, 'Short Track Speed Skating':False, 'Baseball':True,
       'Softball':True, 'Tug-Of-War':True, 'Ski Jumping':False, 'Lacrosse':True, 'Curling':True,
       'Military Ski Patrol':True, 'Cricket':True, 'Croquet':False, 'Motorboating':True,
       'Basque Pelota':False, 'Aeronautics':False, 'Jeu De Paume':False, 'Racquets':False,
       'Roque':False, 'Athletics':False, 'Hockey':True, 'Football':True, 'Wrestling': False, 'Boxing':False, 'Judo':False,
       'Taekwondo':False, 'Shooting':False, 'Weightlifting':False, 'Swimming':False, 'Cycling':False,
       'Alpine Skiing':False, 'Gymnastics':False, 'Fencing':False, 'Handball':True, 'Tennis':False,
       'Volleyball':True, 'Rowing':True, 'Table Tennis':False, 'Trampolining':False,
       'Cross Country Skiing':False, 'Badminton':False, 'Sailing':True, 'Bobsleigh':True,
       'Archery':False, 'Canoeing':False, 'Snowboarding':False, 'Biathlon':False, 'Basketball':True,
       'Beach Volleyball':True, 'Figure Skating':False, 'Polo':True, 'Equestrianism':True,
       'Water Polo':True, 'Art Competitions':False, 'Modern Pentathlon':False, 'Diving':False,
       'Luge':False, 'Freestyle Skiing':False, 'Triathlon':False, 'Skeleton':False,
       'Synchronized Swimming':True, 'Golf':False, 'Rugby Sevens':True
    
}



In [66]:
final_df['TeamGame'] = final_df['Sport'].apply(lambda x: True if sport_dict[x] else False)
# final_df.loc[sport_dict[final_df['Sport']], 'Medal_Bronze'] = 1
# final_df[(final_df.TeamGame == True) & (final_df.Medal_Bronze+final_df.Medal_Silver+final_df.Medal_Gold) > 0]
final_df[(final_df.Sport == 'Basketball') & (final_df.NOC == 'USA')]

,region,Year,NOC,City,Sport,Age,Name,Sex_F,Sex_M,Medal_Bronze,Medal_Silver,Medal_Gold,TeamGame
25459,USA,1936,USA,Berlin,Basketball,24.857143,14,0,14.0,0,0,14,True
25487,USA,1948,USA,London,Basketball,24.428571,14,0,14.0,0,0,14,True
25515,USA,1952,USA,Helsinki,Basketball,23.214286,14,0,14.0,0,0,14,True
25549,USA,1956,USA,Melbourne,Basketball,23.833333,12,0,12.0,0,0,12,True
25568,USA,1960,USA,Roma,Basketball,22.833333,12,0,12.0,0,0,12,True
25603,USA,1964,USA,Tokyo,Basketball,23.500000,12,0,12.0,0,0,12,True
25632,USA,1968,USA,Mexico City,Basketball,23.000000,12,0,12.0,0,0,12,True
25651,USA,1972,USA,Munich,Basketball,20.583333,12,0,12.0,0,12,0,True
25692,USA,1976,USA,Montreal,Basketball,21.000000,24,12,12.0,0,12,12,True
25721,USA,1984,USA,Los Angeles,Basketball,21.833333,24,12,12.0,0,0,24,True


In [32]:
polity_dff2 = prepare_polity_dataset("p5v2018.xls", noc)


In [33]:
country_dict = {'BOSNIA': 'BIH', 'CONGO-BRAZZAVILLE': 'CGO', 'CONGO BRAZZAVILLE': 'CGO',
                    'VIETNAM NORTH': 'VIE', 'TIMOR LESTE': 'TLS', 'GERMANY WEST': 'GER', "COTE D'IVOIRE": 'CIV',
                    'KOREA SOUTH': 'KOR', 'SOUTH VIETNAM': 'VIE', 'SUDAN-NORTH': 'SUD',
                    'TRINIDAD AND TOBAGO': 'TTO', 'UNITED KINGDOM': 'GBR', 'USSR': 'URS', 'YUGOSLAVIA': 'YUG',
                    'SERBIA AND MONTENEGRO': 'YUG', 'YEMEN SOUTH': 'YMD', 'CONGO KINSHASA': 'COD'}
polity_dff3 = map_polity_region_dataset(country_dict, polity_dff2)
